# Medical Question Answering : PubMedQA

Ketrin Mehmeti, Giulia Ghiazza, Leonardo Giorgio Franco, Edoardo Mattei, Alessandro Epifania

### Loading the dataset: 

The object dataset is a DatasetDict, which contains different splits like "train", "validation", and "test" if available.

In [ ]:
pip install pandas pyarrow

#SCARICATE QUESTA LIBRERIA SE NO NON VA UNA MINCHIA

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\213053\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [ ]:
import pandas as pd

# INSERITE LE DIRECTORY DELLE CARTELLE CHE CONTENGONO I FILE .PARQUET


df_artificial = pd.read_parquet(r'C:\Users\213053\OneDrive\Desktop\Magistrale\Natural language processing\Progetto\PubMedQA\pqa_artificial')
df_labeled = pd.read_parquet(r'C:\Users\213053\OneDrive\Desktop\Magistrale\Natural language processing\Progetto\PubMedQA\pqa_labeled')
df_unlabeled = pd.read_parquet(r'C:\Users\213053\OneDrive\Desktop\Magistrale\Natural language processing\Progetto\PubMedQA\pqa_unlabeled')

In [22]:

# Mostra le prime righe
print(df_artificial.head())


      pubid                                           question  \
0  25429730  Are group 2 innate lymphoid cells ( ILC2s ) in...   
1  25433161  Does vagus nerve contribute to the development...   
2  25445714  Does psammaplin A induce Sirtuin 1-dependent a...   
3  25431941  Is methylation of the FGFR2 gene associated wi...   
4  25432519  Do tumor-infiltrating immune cell profiles and...   

                                             context  \
0  {'contexts': ['Chronic rhinosinusitis (CRS) is...   
1  {'contexts': ['Phosphatidylethanolamine N-meth...   
2  {'contexts': ['Psammaplin A (PsA) is a natural...   
3  {'contexts': ['This study examined links betwe...   
4  {'contexts': ['Tumor microenvironment immunity...   

                                         long_answer final_decision  
0  As ILC2s are elevated in patients with CRSwNP,...            yes  
1  Neuronal signals via the hepatic vagus nerve c...            yes  
2  PsA significantly inhibited MCF-7/adr cells pr...    

In [ ]:
print(df_labeled.head())

      pubid                                           question  \
0  21645374  Do mitochondria play a role in remodelling lac...   
1  16418930  Landolt C and snellen e acuity: differences in...   
2   9488747  Syncope during bathing in infants, a pediatric...   
3  17208539  Are the long-term results of the transanal pul...   
4  10808977  Can tailored interventions increase mammograph...   

                                             context  \
0  {'contexts': ['Programmed cell death (PCD) is ...   
1  {'contexts': ['Assessment of visual acuity dep...   
2  {'contexts': ['Apparent life-threatening event...   
3  {'contexts': ['The transanal endorectal pull-t...   
4  {'contexts': ['Telephone counseling and tailor...   

                                         long_answer final_decision  
0  Results depicted mitochondrial dynamics in viv...            yes  
1  Using the charts described, there was only a s...             no  
2  "Aquagenic maladies" could be a pediatric form...    

In [24]:
print(df_unlabeled.head())

      pubid                                           question  \
0  14499029  Is naturopathy as effective as conventional th...   
1  14499049  Can randomised trials rely on existing electro...   
2  14499672  Is laparoscopic radical prostatectomy better t...   
3  14499773  Does bacterial gastroenteritis predispose peop...   
4  14499777  Is early colonoscopy after admission for acute...   

                                             context  \
0  {'contexts': ['Although the use of alternative...   
1  {'contexts': ['To estimate the feasibility, ut...   
2  {'contexts': ['To compare morbidity in two gro...   
3  {'contexts': ['Irritable bowel syndrome (IBS) ...   
4  {'contexts': ['Urgent colonoscopy has been pro...   

                                         long_answer  
0  Naturopathy appears to be an effective alterna...  
1  Routine data have the potential to support hea...  
2  The results of our non-randomized study show t...  
3  Symptoms consistent with IBS and functional